# Installation Notes for K8s and cnvrg Core Locally on M1 and WSL with Helm

## Prerequisites
```
# Download and install Docker Desktop and enable K8s cluster in it
# or install a minikube Cluster
# https://app.cnvrg.io/docs/guides/minikube.html#prerequisites-prepare-your-local-environment
```

Notes:
- The cluster components are created but I was not able to run this

## Install in K8s locally
https://app.cnvrg.io/docs/guides/operator.html
https://helm.sh/docs/intro/install/
https://app.cnvrg.io/docs/guides/helm-light-install.html
```

# In case of minikube - start
# In Docker Desktop setup proper resources pool, like 10 cpus, 20g of RAM
# Remove k8s support in Docker Desktop (there may be a conflict for kubectl)
$ minikube config set EmbedCerts true
$ minikube start --cpus=10 --memory='20g' --disk-size='20g'
# In case of minikube - end


# In case of WSL Ubuntu - start
# Install helm
$ curl https://baltocdn.com/helm/signing.asc | sudo apt-key add -
$ sudo apt-get install apt-transport-https --yes
$ echo "deb https://baltocdn.com/helm/stable/debian/ all main" | sudo tee /etc/apt/sources.list.d/helm-stable-debian.list
$ sudo apt-get update
$ sudo apt-get install helm
# In case of WSL Ubuntu - end

# In case of MacOS - start
# Install and initialize Helm
$ brew install helm
# In case of MacOS - end

# Check if we can connect to a k8s cluster
$ kubectl get nodes

$ helm repo add cnvrg https://charts.cnvrg.io
$ helm repo update


# Get K8s cluster IP
$ kubectl describe node docker-desktop
# 192.168.65.4

# Run the Helm Chart and deploy cnvrg - not working locally
$ helm install cnvrg cnvrg/cnvrg -n cnvrg \
    --create-namespace --timeout 1500s --wait \
    --set clusterDomain=app.192.168.65.4.nip.io
    
# Run the Helm Chart and deploy cnvrg - not working 
# but seems to be the best from different options tried
# should work with minikube
$ helm install cnvrg cnvrg/cnvrg --timeout 1500s --wait --set global.external_ip=$(minikube ip) --set global.node=minikube --set global.high_availability=false -n cnvrg --create-namespace

# In another tab check the status
$ kubectl -n cnvrg get pods
# Or
$ brew install watch
$ watch -c kubectl -n cnvrg get po

# Get IP adress of the cnvrg cluster
kubectl -n cnvrg get svc | grep ingress | awk "{print $4}"


# Monitoring your cluster using Kibana and Grafana
# To access Kibana, go to: kibana.<your_domain>.com
# https://www.elastic.co/guide/en/kibana/7.6/index.html
# https://grafana.com/docs/grafana/latest/
# To access Grafana, go to: grafana.<your_domain>.com


# Upgrade a cnvrg Installation
helm upgrade cnvrg cnvrg/cnvrg --reuse-values \
  --set cnvrgApp.image=<image>


# Delete cnvrg CORE
$ helm uninstall cnvrg -n cnvrg
```